In [44]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [45]:
from openai import OpenAI
GPT_MODEL = "gpt-4o-mini"
client = OpenAI(api_key=OPENAI_API_KEY)

assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model=GPT_MODEL,
)

In [46]:
thread = client.beta.threads.create()

In [47]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [48]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    """
    Refers to any token creation essentially.
    """
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = sp.Eq(3*x + 11, 14)

# Solve the equation
solution = sp.solve(equation, x)
solution
assistant > The solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).

In [50]:
for message in client.beta.threads.messages.list(thread.id):
    print(message.content)

[TextContentBlock(text=Text(annotations=[], value='The solution to the equation \\( 3x + 11 = 14 \\) is \\( x = 1 \\).'), type='text')]
[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')]
